# Naive Bayes Binary Classification

In [1]:
# Importing the library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # visualization library based on matplotlib
from IPython.display import display # Manage multiple output per cell

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.classification import log_loss
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# Manage UTF-8 issue
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

import datetime

In [3]:
# DB Sqlite connection
import sqlite3
db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
conn = sqlite3.connect(db)
cur = conn.cursor()

# Features List

In [4]:
all_features = ["A_FIVE_AC","A_FIVE_AF","A_FIVE_AR","A_FIVE_AS","A_FIVE_AST","A_FIVE_AY","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HF","A_FIVE_HR","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","A_FIVE_HY","H_FIVE_AC","H_FIVE_AF","H_FIVE_AR","H_FIVE_AS","H_FIVE_AST","H_FIVE_AY","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HF","H_FIVE_HR","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","H_FIVE_HY","A_THREE_AC","A_THREE_AF","A_THREE_AR","A_THREE_AS","A_THREE_AST","A_THREE_AY","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HF","A_THREE_HR","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","A_THREE_HY","H_THREE_AC","H_THREE_AF","H_THREE_AR","H_THREE_AS","H_THREE_AST","H_THREE_AY","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HF","H_THREE_HR","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","H_THREE_HY","INFO_Div"] 
three_matches_features = ["A_THREE_AC","A_THREE_AF","A_THREE_AR","A_THREE_AS","A_THREE_AST","A_THREE_AY","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HF","A_THREE_HR","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","A_THREE_HY","H_THREE_AC","H_THREE_AF","H_THREE_AR","H_THREE_AS","H_THREE_AST","H_THREE_AY","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HF","H_THREE_HR","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","H_THREE_HY","INFO_Div"]
five_matches_features = ["A_FIVE_AC","A_FIVE_AF","A_FIVE_AR","A_FIVE_AS","A_FIVE_AST","A_FIVE_AY","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HF","A_FIVE_HR","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","A_FIVE_HY","H_FIVE_AC","H_FIVE_AF","H_FIVE_AR","H_FIVE_AS","H_FIVE_AST","H_FIVE_AY","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HF","H_FIVE_HR","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","H_FIVE_HY","INFO_Div"]
all_features_no_div = ["A_FIVE_AC","A_FIVE_AF","A_FIVE_AR","A_FIVE_AS","A_FIVE_AST","A_FIVE_AY","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HF","A_FIVE_HR","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","A_FIVE_HY","H_FIVE_AC","H_FIVE_AF","H_FIVE_AR","H_FIVE_AS","H_FIVE_AST","H_FIVE_AY","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HF","H_FIVE_HR","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","H_FIVE_HY","A_THREE_AC","A_THREE_AF","A_THREE_AR","A_THREE_AS","A_THREE_AST","A_THREE_AY","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HF","A_THREE_HR","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","A_THREE_HY","H_THREE_AC","H_THREE_AF","H_THREE_AR","H_THREE_AS","H_THREE_AST","H_THREE_AY","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HF","H_THREE_HR","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","H_THREE_HY"] 
three_matches_features_no_div = ["A_THREE_AC","A_THREE_AF","A_THREE_AR","A_THREE_AS","A_THREE_AST","A_THREE_AY","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HF","A_THREE_HR","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","A_THREE_HY","H_THREE_AC","H_THREE_AF","H_THREE_AR","H_THREE_AS","H_THREE_AST","H_THREE_AY","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HF","H_THREE_HR","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","H_THREE_HY"]
five_matches_features_no_div = ["A_FIVE_AC","A_FIVE_AF","A_FIVE_AR","A_FIVE_AS","A_FIVE_AST","A_FIVE_AY","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HF","A_FIVE_HR","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","A_FIVE_HY","H_FIVE_AC","H_FIVE_AF","H_FIVE_AR","H_FIVE_AS","H_FIVE_AST","H_FIVE_AY","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HF","H_FIVE_HR","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","H_FIVE_HY"]
less_features = ["A_FIVE_AC","A_FIVE_AS","A_FIVE_AST","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","H_FIVE_AC","H_FIVE_AS","H_FIVE_AST","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","A_THREE_AC","A_THREE_AS","A_THREE_AST","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","H_THREE_AC","H_THREE_AS","H_THREE_AST","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","INFO_Div"]
less_features_no_div = ["A_FIVE_AC","A_FIVE_AS","A_FIVE_AST","A_FIVE_FTAG","A_FIVE_FTHG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HC","A_FIVE_HS","A_FIVE_HST","A_FIVE_HTAG","A_FIVE_HTHG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","H_FIVE_AC","H_FIVE_AS","H_FIVE_AST","H_FIVE_FTAG","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTAG","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","A_THREE_AC","A_THREE_AS","A_THREE_AST","A_THREE_FTAG","A_THREE_FTHG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HC","A_THREE_HS","A_THREE_HST","A_THREE_HTAG","A_THREE_HTHG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","H_THREE_AC","H_THREE_AS","H_THREE_AST","H_THREE_FTAG","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HS","H_THREE_HST","H_THREE_HTAG","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H"] 
home_away_features = ["A_FIVE_AC","A_FIVE_AS","A_FIVE_AST","A_FIVE_FTAG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HTAG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","A_THREE_AC","A_THREE_AS","A_THREE_AST","A_THREE_FTAG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HTAG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HS","H_THREE_HST","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H","INFO_Div"]
home_away_features_no_div = ["A_FIVE_AC","A_FIVE_AS","A_FIVE_AST","A_FIVE_FTAG","A_FIVE_FTR_A","A_FIVE_FTR_D","A_FIVE_FTR_H","A_FIVE_HTAG","A_FIVE_HTR_A","A_FIVE_HTR_D","A_FIVE_HTR_H","H_FIVE_FTHG","H_FIVE_FTR_A","H_FIVE_FTR_D","H_FIVE_FTR_H","H_FIVE_HC","H_FIVE_HS","H_FIVE_HST","H_FIVE_HTHG","H_FIVE_HTR_A","H_FIVE_HTR_D","H_FIVE_HTR_H","A_THREE_AC","A_THREE_AS","A_THREE_AST","A_THREE_FTAG","A_THREE_FTR_A","A_THREE_FTR_D","A_THREE_FTR_H","A_THREE_HTAG","A_THREE_HTR_A","A_THREE_HTR_D","A_THREE_HTR_H","H_THREE_FTHG","H_THREE_FTR_A","H_THREE_FTR_D","H_THREE_FTR_H","H_THREE_HC","H_THREE_HS","H_THREE_HST","H_THREE_HTHG","H_THREE_HTR_A","H_THREE_HTR_D","H_THREE_HTR_H"]

# Comparaison Table

In [5]:
def computeModelEstimation(league, test_season, training_year, odds, features_list, features_list_title, model):
    # The result that will be return
    result = {}
    result['model'] = model
    result['league'] = league
    result['test_season'] = str(test_season)+'/'+str(test_season+1)
    result['nb_training_season'] = training_year
    result['odds'] = odds
    result['features_list'] = features_list_title
    # Extract odds
    lowest_odd = float(odds.split('-')[0])
    highest_odd = float(odds.split('-')[1])
    # Filter query by date
    query_start_date = str(test_season-training_year)+'-10-01'
    query_end_date = str(test_season+1)+'-06-01'
    # Query the dataset
    df = pd.read_sql_query("SELECT * FROM pre_matchs WHERE INFO_Date > '"+query_start_date+"' AND INFO_Date < '"+query_end_date+"' ORDER BY INFO_Date ASC;", conn)
    # Remove the index column
    df = (df[df.columns.drop(['index'])])
    # Remove all game between June (include) and October (include)
    df['INFO_Date'] = pd.to_datetime(df['INFO_Date'])
    df['INFO_Date'].dt.month
    df = df[(df['INFO_Date'].dt.month < 6) | (df['INFO_Date'].dt.month > 10)]
    # Filter dataset by league
    if league != 'ALL':
        df = df[(df['INFO_Div'] == league)]
    # Filter dataset by odd
    df = df.drop(df[df.INFO_BbAvA < lowest_odd].index)
    df = df.drop(df[df.INFO_BbAvA > highest_odd].index)
    # Create a INFO_WIN column containing the gain. If bet success it's equal to odd -1, else -1 (lose you bet)
    df['INFO_WIN'] = df['INFO_BbAvA']-1
    df.loc[df.INFO_FTR != 'A', 'INFO_WIN'] = -1
    # Keep season test_season out of training set
    date_start_current_season = datetime.date(test_season, 10, 1)
    df_current_season = df[(df['INFO_Date'] > date_start_current_season)]
    df = df[(df['INFO_Date'] < date_start_current_season)]
    # How much did you lose per match in the season if bet on all
    result['naive_bet'] = df_current_season.INFO_WIN.mean()
    # Prepare X and y
    X = pd.get_dummies(df[features_list])
    y = pd.get_dummies(df)['INFO_FTR_A'].astype('bool_')
    X_current_season = pd.get_dummies(df_current_season[features_list])
    y_current_season = pd.get_dummies(df_current_season)['INFO_FTR_A'].astype('bool_')
    # Standardize features
    sc_X = StandardScaler().fit(X)
    X = sc_X.transform(X)
    X_current_season = sc_X.transform(X_current_season)
    # Impute of missing values (NaN) with the mean
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp = imp.fit(X)
    X = imp.transform(X)
    X_current_season = imp.transform(X_current_season)
    # Split the dataset into the Training set and Test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)
    # Import Naive Bayes Classifier
    classifier = GaussianNB()
    # Get best parameters using Grid Search
    parameters = [{}]
    clf = GridSearchCV(estimator=classifier,
                              param_grid=parameters,
                              scoring='f1',
                              cv=8,
                              n_jobs=-1)
    clf.fit(X_train, y_train)
    result['best_score'] = clf.best_score_
    # Predict target values
    y_test_pred = clf.predict(X_test)
    # Predict probabilities
    y_test_probs = clf.predict_proba(X_test)
    # Compute precision, recall, F-measure and support
    test_scores = precision_recall_fscore_support(y_test, y_test_pred, average='binary')
    result['y_test_precision'] = test_scores[0]
    result['y_test_recall'] = test_scores[1]
    result['y_test_fscore'] = test_scores[2]
    # ALL DATASET TEST
    X_pred = clf.predict(X)
    df['pred'] = X_pred
    df_bet_home = df.drop(df[df.pred == 0].index)
    X_scores = precision_recall_fscore_support(y, X_pred, average='binary') 
    result['X_precision'] = X_scores[0]
    result['X_recall'] = X_scores[1]
    result['X_fscore'] = X_scores[2]
    result['X_money'] = df_bet_home.INFO_WIN.mean()
    # SEASON TEST
    X_pred_current_season = clf.predict(X_current_season)
    df_current_season['pred'] = X_pred_current_season
    df_bet_home_current_season = df_current_season.drop(df_current_season[df_current_season.pred == 0].index)
    season_scores = precision_recall_fscore_support(y_current_season, X_pred_current_season, average='binary') 
    result['season_precision'] = season_scores[0]
    result['season_recall'] = season_scores[1]
    result['season_fscore'] = season_scores[2]
    result['season_money'] = df_bet_home_current_season.INFO_WIN.mean()
    result['season_nb_bets'] = df_bet_home_current_season.shape[0]
    return result

In [6]:
# Create a CSV comparaison file
features_title = 'home_away_features_no_div'
features = home_away_features_no_div
# On which league the model must be training (ALL means all leagues are mixed together)
leagues = ['ALL', 'D1', 'E0', 'E1', 'E2', 'F1', 'I1', 'SP1', 'SC0']
#leagues = ['ALL', 'D1']
# Which season will be the test season (year correspond to the start of the season)
#test_seasons = [2016, 2015, 2014]
test_seasons = [2016, 2015, 2014, 2013, 2012, 2011]
#test_seasons = [2016]
# On how many past seasons we train the data
#training_years = [1, 2, 3, 4, 5]
training_years = [1, 2]
# On which odd are we authorised to bet
#odd_ranges = ['1-10', '1.5-3', '1.5-4', '1.5-4.5', '1.5-5', '1.8-3', '1.8-4', '1.8-4.5', '1.8-5', '2-3', '2-4', '2-4.5', '2-5', '2.2-3', '2.2-4', '2.2-4.5', '2.2-5'] 
odd_ranges = ['1-10','1.8-4', '1.8-4.5', '1.8-5', '2-4', '2-4.5', '2-5', '2.2-4', '2.2-4.5', '2.2-5'] 
# Dataframe with result comparaison
result = []
# Test all combination
for training_year in training_years:
    display(training_year)
    for test_season in test_seasons:
        display(test_season)
        for odd_range in odd_ranges:
            display(odd_range)
            for league in leagues:
                display(league)
                result_dic = computeModelEstimation(league, test_season, training_year, odd_range, features, features_title, 'GaussianNB')
                result.append(result_dic)
comparaison_df = pd.DataFrame(result)

1

2016

'1-10'

'ALL'

'D1'

...

'E2'

'F1'

'I1'

'SP1'

'SC0'

In [7]:
comparaison_df.to_csv("/Users/thibaultclement/Project/machine-learning/data/result/FTR-Away_Naive_Bayes_Comparaison-"+features_title+".csv")